# 使用 Facebook fastText 工具对数据集进行情感分析

使用1-5的评分，任务是预测评论的评分。使用fastText的监督学习方法，希望模型可以将评论分类为5个评分之一。

## 安装 fasttext

从 git 下载 fasttext 并使用 pip 安装。

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip

In [ ]:
!unzip v0.9.2.zip

In [ ]:
!cd fastText-0.9.2 && pip install .

## 清理数据

作为使用不同类型数据集的 fasttext，与 TF 或 Keras 不同。Fasttext 使用基于文本的文件数据集，标签与数据位于同一行。标签必须以`__label__<label word>` 开头，因此需要更改当前数据集。
    
当然，另一个原因是当前数据集仍然处于“脏”状态，带有混合大小写字符串，尚未进行流形化或词干化，还包含新行和其他内容。
   

In [ ]:
# "The cleaning supply"
import re
import string

# "NLP Supply"
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

打开文件、主要训练数据以及测试数据。

In [ ]:
train = pd.read_csv("../input/student-shopee-code-league-sentiment-analysis/train.csv")
test = pd.read_csv("../input/student-shopee-code-league-sentiment-analysis/test.csv")

检查两个数据集的内部

In [ ]:
train.head()

In [ ]:
test.head()

用于预处理文本的 Cleaner 函数。

In [ ]:
# Initialize the lemmarizer and stemmer
wordnet_lemmatizer = WordNetLemmatizer()
englishStemmer = SnowballStemmer("english")


def clean_text(x):
    # Remove zero width space from the string and lower it
    temp_text = x.lower().replace("\u200b", "")
    # Remove punctuation of the string
    temp_text = temp_text.translate(str.maketrans("", "", string.punctuation))
    # Remove new line from string
    temp_text = temp_text.replace("\n", "")
    # Remove double space or more
    temp_text = re.sub(" +", " ", temp_text).strip()
    # Tokenized the text
    temp_text = nltk.word_tokenize(temp_text)
    stop_words = set(stopwords.words("english"))

    filtered_word = []

    for word in temp_text:
        # Lemmanize and stem word
        lemma_word = wordnet_lemmatizer.lemmatize(word)
        stemmed_word = englishStemmer.stem(lemma_word)

        # Do not add stop words into the the final cleaned sentence
        if stemmed_word in stop_words:
            continue
        else:
            filtered_word.append(stemmed_word)

    return " ".join(filtered_word).strip()

In [ ]:
# Clean all of the review in training, and test
train["review"] = train["review"].apply(clean_text)
test["review"] = test["review"].apply(clean_text)

将训练和验证数据拆分为 2 个不同的数据帧。

In [ ]:
VAL_PERCENTAGE = 0.2
N_VAL = int(len(train) * VAL_PERCENTAGE)

# Shuffle train DataFrame also reset the shuffled index
train = train.sample(frac=1).reset_index(drop=True)

# Set validation DataFrame as having the first N_VAL row
val_data = train[:N_VAL]

# Set train DataFrame as the rest
train_data = train[N_VAL:]

In [ ]:
val_data.describe()

In [ ]:
train_data.describe()

## 转换数据集

创建函数将当前数据集行转换为字符串，该字符串是fastText的训练数据中的行。
它看起来像 `__label_<1-5><review>`。

In [ ]:
validation_file_path = "./review.val"
training_file_path = "./review.train"


def append_fasttext_dataset(row, file_writer):
    def convert_row_to_dataset_string(row):
        return "__label__" + str(row["rating"]) + " " + row["review"]

    file_writer.write(convert_row_to_dataset_string(row) + "\n")

写入验证和训练文件

In [ ]:
# Validation file
with open(validation_file_path, "a+") as writer:
    val_data.apply(lambda x: append_fasttext_dataset(x, writer), axis=1)

# Training file
with open(training_file_path, "a+") as writer:
    train_data.apply(lambda x: append_fasttext_dataset(x, writer), axis=1)

查看数据集，看看是否存在偏差或任何其他差异。

In [ ]:
# Look at number of label (1-5) of the training data
train_data["rating"].value_counts().plot.bar()

In [ ]:
# Look at number of label (1-5) of the validation data
val_data["rating"].value_counts().plot.bar()

如条形图所示，数据偏向较高的评级。这对于评论数据来说是常见的，因为大多数人可能只是在他们的购买上留下了一个好分数，除非发生了小事情，它将是4星，有时是3星。但如果这是一次重大的采购事件，人们只会给卖家一颗星的评价。

这对培训来说是一件坏事，因为模型（可能）更可能在审查中给出比实际情况更高的评级。使用上采样方法，通过将少数类复制到与多数类相同或几乎相同的级别，可以解决数据集的这个问题。这样做的问题是，我们将冒风险，过度拟合模型，重复审查。特别是前面所示的巨大差异。

现在，我们将“按原样”运行它，看看会发生什么。

## 使用 fastText 进行训练

现在，实际制作模型的有趣部分将对我们的数据进行分类。我们将使用fastText的监督训练方法来完成。

In [ ]:
import fasttext

定义训练阶段的参数，

- **lr** : 训练的学习率
- **epoch** : 训练将对数据进行多少次检查
- **wordNgrams** : 给定文本样本中最大 n 个单词的连续序列
- **dim** : 词向量的维数

In [ ]:
hyper_params = {"lr": 0.01, "epoch": 15, "wordNgrams": 2, "dim": 20, "verbose": 1}

现在，使用适当的输入文件构建我们将使用的fasttext模型，然后模型将使用我们的参数自动使用该文件进行训练。

In [ ]:
model = fasttext.train_supervised(input=training_file_path, **hyper_params)

检查模型本身的性能

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Get model accuracy and accuracy of the validation
result = model.test(training_file_path)
validation = model.test(validation_file_path)

print("Result : ", result)
print("Validation : ", validation)

# Plot the result
accuracy_data = [result[1], validation[1]]
labels = ["Model Accuracy", "Validation Accuracy"]

plt.title("Model accuracy")
plt.bar(labels, accuracy_data)
plt.show()

该模型的准确率在50%以上，验证数据也显示了这一点（稍微有点差异是可以的），但让我们看看如何实际改进它。也许首先我们需要查看模型的混淆矩阵，看看它是如何处理标签的，以及哪个标签实际上对它影响最大。

为了对数据帧中使用的标签进行预测和转换，我们将创建一个函数。

In [ ]:
def get_predicted_rating(x, model):
    return int(model.predict(x)[0][0].split("__label__")[1])

现在获得所有验证数据的预测结果。

In [ ]:
val_data["predicted"] = val_data["review"].apply(
    lambda x: get_predicted_rating(x, model)
)

然后使用 sklearn 工具制作混淆矩阵，以更轻松地解释数据。

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_labels = [1, 2, 3, 4, 5]
confusion_matrix_data = confusion_matrix(
    val_data["rating"], val_data["predicted"], labels=confusion_labels
)
normalised_confusion_matrix = (
    confusion_matrix_data.astype("float")
    / confusion_matrix_data.sum(axis=1)[:, np.newaxis]
)

In [ ]:
# Plot the normalised confusion matrix
ax = plt.subplot()
sns.heatmap(normalised_confusion_matrix, annot=True, ax=ax, fmt=".2f")

ax.set_title("Normalized Confusion Matrix")

ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")

ax.xaxis.set_ticklabels(confusion_labels)
ax.yaxis.set_ticklabels(confusion_labels)

看来我们的预测是错误的！大多数问题存在于等级4和等级5之间的假阳性和假阴性之间。等级3和等级2之间也存在假阳性。所有这些标签的值均为0.4-ish，这意味着几乎一半的标签被错误地标记为其相应的相邻等级。这可能意味着评级4和5、5和4，以及评级3和2具有密切相关的趋势，或密切正或负。

## 探索改进
在本节中，很少有人尝试使用不同的方法来改进当前模型。

### 更多的 Epoch
通过将模型更多地暴露在数据中，模型可能会更好地“学习”模式，并更好地区分数据中的模式。

首先，让我们确定执行此操作的参数。让我们试着再加一点，也许10。

In [ ]:
hyper_params_25_epoch = {
    "lr": 0.01,
    "epoch": 25,
    "wordNgrams": 2,
    "dim": 20,
    "verbose": 1,
}

In [ ]:
model_25_epoch = fasttext.train_supervised(
    input=training_file_path, **hyper_params_25_epoch
)

In [ ]:
# Get model accuracy and accuracy of the validation
result = model_25_epoch.test(training_file_path)
validation = model_25_epoch.test(validation_file_path)

print("Result : ", result)
print("Validation : ", validation)

# Plot the result
accuracy_data = [result[1], validation[1]]
labels = ["Model Accuracy", "Validation Accuracy"]

plt.title("Model accuracy with 25 epoch")
plt.bar(labels, accuracy_data)
plt.show()

模型精度本身（使用训练数据）与验证数据精度之间的差异似乎比以前有更大的差距。这可能意味着模型正在开始或已经过拟合，因为它非常了解训练数据，但不知道它从未见过的数据。

### 更大的 Learning Rate

通过对预测误差率做出更大的响应，模型可能会更快地找到“谷”，并最终使其更准确。

现在，使超参数具有更高的学习率。我可能会选择，0.6或什么。

In [ ]:
hyper_params_bigger_lr = {
    "lr": 0.6,
    "epoch": 15,
    "wordNgrams": 2,
    "dim": 20,
    "verbose": 1,
}

In [ ]:
model_bigger_lr = fasttext.train_supervised(
    input=training_file_path, **hyper_params_bigger_lr
)

In [ ]:
# Get model accuracy and accuracy of the validation
result = model_bigger_lr.test(training_file_path)
validation = model_bigger_lr.test(validation_file_path)

print("Result : ", result)
print("Validation : ", validation)

# Plot the result
accuracy_data = [result[1], validation[1]]
labels = ["Model Accuracy", "Validation Accuracy"]

plt.title("Model accuracy with learning rate 0.6")
plt.bar(labels, accuracy_data)
plt.show()

看起来使用更高的学习率也没有帮助，因为训练数据和验证数据的差异越来越大，显示出过度拟合。

### Autotuning

尝试使用作为参数提供的验证数据使模型实际改进自身。

现在，为它创建参数。通过传递验证文件，模型将使用该文件作为模型的优化参考。

In [ ]:
hyper_params_autotuning = {
    "lr": 0.06,
    "epoch": 20,
    "wordNgrams": 2,
    "dim": 20,
    "verbose": 1,
    "autotuneValidationFile": validation_file_path,
}

In [ ]:
model_autotuning = fasttext.train_supervised(
    input=training_file_path, **hyper_params_autotuning
)

In [ ]:
# Get model accuracy and accuracy of the validation
result = model_autotuning.test(training_file_path)
validation = model_autotuning.test(validation_file_path)

print("Result : ", result)
print("Validation : ", validation)

# Plot the result
accuracy_data = [result[1], validation[1]]
labels = ["Model Accuracy", "Validation Accuracy"]

plt.title("Model accuracy with autotuning")
plt.bar(labels, accuracy_data)
plt.show()

In [ ]:
val_data["predicted"] = val_data["review"].apply(
    lambda x: get_predicted_rating(x, model_autotuning)
)

In [ ]:
confusion_labels = [1, 2, 3, 4, 5]
confusion_matrix_data = confusion_matrix(
    val_data["rating"], val_data["predicted"], labels=confusion_labels
)
normalised_confusion_matrix = (
    confusion_matrix_data.astype("float")
    / confusion_matrix_data.sum(axis=1)[:, np.newaxis]
)

In [ ]:
# Plot the normalised confusion matrix
ax = plt.subplot()
sns.heatmap(normalised_confusion_matrix, annot=True, ax=ax, fmt=".2f")

ax.set_title("Normalized Confusion Matrix (Autotuning)")

ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")

ax.xaxis.set_ticklabels(confusion_labels)
ax.yaxis.set_ticklabels(confusion_labels)

自动调谐的使用似乎令人惊讶，在没有太多过度拟合的情况下达到了更高的精度，尽管在混乱矩阵上，一切都像以前一样，没有任何变化。在autotune函数中，我们还可以更改一些指标，例如优化某个标签的f1分数。

这就是我要做的。让我们看看在优化 `__label__4` 的f1分数时使用聚焦指标会发生什么。

In [ ]:
hyper_params_autotuning_metrics = {
    "lr": 0.1,
    "epoch": 20,
    "wordNgrams": 2,
    "dim": 50,
    "verbose": 2,
    "autotuneValidationFile": validation_file_path,
    "autotuneMetric": "f1:__label__4",
}

In [ ]:
model_autotuning_metrics = fasttext.train_supervised(
    input=training_file_path, **hyper_params_autotuning_metrics
)

In [ ]:
# Get model accuracy and accuracy of the validation
result = model_autotuning_metrics.test(training_file_path)
validation = model_autotuning_metrics.test(validation_file_path)

print("Result : ", result)
print("Validation : ", validation)

# Plot the result
accuracy_data = [result[1], validation[1]]
labels = ["Model Accuracy", "Validation Accuracy"]

plt.title("Model accuracy with autotuning")
plt.bar(labels, accuracy_data)
plt.show()

In [ ]:
val_data["predicted"] = val_data["review"].apply(
    lambda x: get_predicted_rating(x, model_autotuning_metrics)
)

In [ ]:
confusion_labels = [1, 2, 3, 4, 5]
confusion_matrix_data = confusion_matrix(
    val_data["rating"], val_data["predicted"], labels=confusion_labels
)
normalised_confusion_matrix = (
    confusion_matrix_data.astype("float")
    / confusion_matrix_data.sum(axis=1)[:, np.newaxis]
)

In [ ]:
# Plot the normalised confusion matrix
ax = plt.subplot()
sns.heatmap(normalised_confusion_matrix, annot=True, ax=ax, fmt=".2f")

ax.set_title("Normalized Confusion Matrix (Autotuning Metrics f1:__label__4)")

ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")

ax.xaxis.set_ticklabels(confusion_labels)
ax.yaxis.set_ticklabels(confusion_labels)

按照预期进行度量的结果是，评级4和5之间的混淆已经减少，尽管评级5和4之间的混淆正在增加。另一个未预料到但受欢迎的影响是评级2和3的混淆率较低。

现在，如果重点放在评级5上呢。

In [ ]:
hyper_params_autotuning_metrics_5 = {
    "lr": 0.1,
    "epoch": 20,
    "wordNgrams": 2,
    "dim": 50,
    "verbose": 2,
    "autotuneValidationFile": validation_file_path,
    "autotuneMetric": "f1:__label__5",
}

In [ ]:
model_autotuning_metrics_5 = fasttext.train_supervised(
    input=training_file_path, **hyper_params_autotuning_metrics_5
)

In [ ]:
# Get model accuracy and accuracy of the validation
result = model_autotuning_metrics_5.test(training_file_path)
validation = model_autotuning_metrics_5.test(validation_file_path)

print("Result : ", result)
print("Validation : ", validation)

# Plot the result
accuracy_data = [result[1], validation[1]]
labels = ["Model Accuracy", "Validation Accuracy"]

plt.title("Model accuracy with autotuning metrics")
plt.bar(labels, accuracy_data)
plt.show()

In [ ]:
val_data["predicted"] = val_data["review"].apply(
    lambda x: get_predicted_rating(x, model_autotuning_metrics_5)
)

In [ ]:
confusion_labels = [1, 2, 3, 4, 5]
confusion_matrix_data = confusion_matrix(
    val_data["rating"], val_data["predicted"], labels=confusion_labels
)
normalised_confusion_matrix = (
    confusion_matrix_data.astype("float")
    / confusion_matrix_data.sum(axis=1)[:, np.newaxis]
)

In [ ]:
# Plot the normalised confusion matrix
ax = plt.subplot()
sns.heatmap(normalised_confusion_matrix, annot=True, ax=ax, fmt=".2f")

ax.set_title("Normalized Confusion Matrix (Autotuning Metrics f1:__label__5)")

ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")

ax.xaxis.set_ticklabels(confusion_labels)
ax.yaxis.set_ticklabels(confusion_labels)

这不会产生比使用标签5的方法更好的混淆矩阵

## 提交文件
按照预期的样本提交格式，制作提交给kaggle的文件。

In [ ]:
submission = test.copy()
submission["rating"] = submission["review"].apply(
    lambda x: get_predicted_rating(x, model_autotuning_metrics)
)

del submission["review"]

查看预测结果。

In [ ]:
submission.head()

将提交文件保存到csv中，csv文件中没有索引。

In [ ]:
submission.to_csv("submission.csv", index=False)